In [47]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc

from datetime import datetime
import requests
import json
from pandas import json_normalize

from tqdm import tqdm
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston

In [48]:
today = datetime.today().strftime('%Y%m%d')
year_list =[2020, 2021, 2022]
df_ho =pd.DataFrame([])
for i in year_list:
    today_year = i

    key = 'UhnjyvmWEq5SH4yCqPylCPQSp4NdA524up%2FM1CrsUf3143Vmj3Rwx7PGHxq5YZY4mFHNHQ%2F0ixGwapRFApPsaw%3D%3D'
    url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?_type=json&numOfRows=50&solYear=' + str(today_year) + '&ServiceKey=' + str(key)
    response = requests.get(url)
    if response.status_code == 200:
        json_ob = json.loads(response.text)
        holidays_data = json_ob['response']['body']['items']['item']
        dataframe = json_normalize(holidays_data)
    df_ho = pd.concat([df_ho, dataframe])

In [49]:
df_ho=df_ho.reset_index()

In [50]:
df_ho['locdate']=df_ho['locdate'].reset_index(drop=True)

In [51]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [52]:
csv_to_parquet('C:/Users/gisic/gisic/jejutraffic/train.csv', 'train')
csv_to_parquet('C:/Users/gisic/gisic/jejutraffic/test.csv', 'test')

train Done.
test Done.


In [53]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')

In [54]:
str_col = ['start_turn_restricted','end_turn_restricted']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

In [55]:
t_train = train.copy()
t_test = test.copy()

In [56]:
y = t_train['target'] 

x = t_train.drop(['id','target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted','road_type','weight_restricted','height_restricted','multi_linked','road_rating','connect_code'], axis=1)

t_test = t_test.drop(['id','road_name', 'start_node_name', 'end_node_name','vehicle_restricted','road_type','weight_restricted','height_restricted','multi_linked','road_rating','connect_code'], axis=1)

print(x.shape)
print(y.shape)
print(t_test.shape)

(4701217, 12)
(4701217,)
(291241, 11)


In [59]:
col_list = ['base_hour','lane_count','day_of_week','maximum_speed_limit']
for i in col_list:
    x = pd.get_dummies(x,columns = [i])
    t_test = pd.get_dummies(t_test,columns = [i])

In [61]:
for i in (df_ho['locdate']):
    x.loc[(x['base_date'] == i),'base_date'] = 1
x.loc[(x['base_date'] > 1),'base_date'] = 0

for i in (df_ho['locdate']):
    t_test.loc[(t_test['base_date'] == i),'base_date'] = 1
    
t_test.loc[(t_test['base_date'] > 1),'base_date'] = 0    

100%|████████████████████████████████████████████████████████████████████████████████| 55/55 [00:00<00:00, 1227.91it/s]


In [63]:
from sklearn.preprocessing import OneHotEncoder

In [99]:
A_cnt = x['start_longitude'].value_counts()
print("road_name :\n", x['start_longitude'].value_counts())
print()

road_name :
 126.567766    23291
126.736249    20775
126.456384    18137
126.556173    18084
126.505998    18076
              ...  
126.682267      881
126.677154      788
126.524410      744
126.524429      744
126.525745      587
Name: start_longitude, Length: 586, dtype: int64



In [128]:
x_sample = x.iloc[:100,:]

In [153]:
x_sample

,base_date,start_latitude,start_longitude,start_turn_restricted,end_latitude,end_longitude,end_turn_restricted,target,base_hour_0,base_hour_1,...,x1_76174,x1_77658,x1_78635,x1_78816,x1_80766,x1_81315,x1_82565,x1_87731,x1_89611,x1_97078
0,0,33.427747,126.662612,0,33.427749,126.662335,0,52.0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,33.500730,126.529107,1,33.504811,126.526240,0,30.0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,33.279145,126.368598,0,33.280072,126.362147,0,61.0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,33.246081,126.567204,0,33.245565,126.566228,0,20.0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,33.462214,126.326551,0,33.462677,126.330152,0,38.0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,33.247388,126.568642,0,33.248505,126.569797,0,23.0,0,0,...,0,0,0,0,1,0,0,0,0,0
96,0,33.390187,126.485480,0,33.391210,126.483575,0,39.0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,1,33.248491,126.566092,1,33.247607,126.561943,0,21.0,0,0,...,0,0,0,0,0,1,0,0,0,0
98,0,33.519338,126.874347,0,33.518311,126.861000,0,48.0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [130]:
x_sample['x1'] = 10000*(round((x_sample['start_latitude'] * x_sample['start_longitude']),4)-4200)
x_sample['x1']= x_sample['x1'].astype('int')
ohe = OneHotEncoder()
sample_latitude = ohe.fit_transform(x[['x1']])

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_12128\3957826651.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_sample['x1'] = 10000*(round((x_sample['start_latitude'] * x_sample['start_longitude']),4)-4200)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_12128\3957826651.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_sample['x1']= x_sample['x1'].astype('int')


In [132]:
sample_cnt = x_sample['x1'].value_counts()
print("road_name :\n", x_sample['x1'].value_counts())
print()

road_name :
 356109    3
325052    2
394279    2
329022    2
97078     2
         ..
236890    1
315131    1
114886    1
336211    1
193859    1
Name: x1, Length: 90, dtype: int64



In [144]:
x_sample['x1']= x_sample['x1'].astype('object')

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_12128\3143975574.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_sample['x1']= x_sample['x1'].astype('object')


In [145]:
x_sample['x1'].dtypes

dtype('O')

In [148]:
ass = (col for col in ohe.categories_[0])

In [158]:
ssa=[]

In [159]:
for col in ohe.categories_[0]:
    print(str(col))
    ssa.append(str(col))

-28850
-25626
-3221
-1710
3912
4233
4408
10808
14575
22438
25999
27020
27876
28631
28855
30061
32445
33311
34481
37532
37610
37749
38015
38095
38685
39326
39417
40227
40518
40690
40780
41067
41163
41266
41487
43531
45257
45621
46790
47043
47542
47734
47975
49943
50001
52755
52941
52955
53362
53398
53414
54389
54987
55082
55405
58442
58625
59311
60246
60438
63068
63281
63301
63339
63967
64327
64476
64771
64832
65334
65547
66030
66049
66058
66068
66087
66750
67263
67299
67335
67353
67857
68005
68409
68460
68512
69632
69660
69687
69911
70119
70847
70868
70878
70888
70892
70909
70914
70973
71001
71760
72259
72806
72842
73076
73168
73716
75057
76174
76229
76809
77276
77587
77658
77807
78635
78742
78757
78816
79174
79488
80014
80766
81288
81315
81634
81643
82052
82565
83631
84377
85006
85599
86161
86350
87070
87731
88248
88539
88757
89236
89611
91082
91257
91373
92151
92174
93482
94975
95519
96157
97078
97174
98774
100268
101193
101286
101683
102283
102436
103806
103904
104147
104346
106737


In [160]:
ssa

['-28850',
 '-25626',
 '-3221',
 '-1710',
 '3912',
 '4233',
 '4408',
 '10808',
 '14575',
 '22438',
 '25999',
 '27020',
 '27876',
 '28631',
 '28855',
 '30061',
 '32445',
 '33311',
 '34481',
 '37532',
 '37610',
 '37749',
 '38015',
 '38095',
 '38685',
 '39326',
 '39417',
 '40227',
 '40518',
 '40690',
 '40780',
 '41067',
 '41163',
 '41266',
 '41487',
 '43531',
 '45257',
 '45621',
 '46790',
 '47043',
 '47542',
 '47734',
 '47975',
 '49943',
 '50001',
 '52755',
 '52941',
 '52955',
 '53362',
 '53398',
 '53414',
 '54389',
 '54987',
 '55082',
 '55405',
 '58442',
 '58625',
 '59311',
 '60246',
 '60438',
 '63068',
 '63281',
 '63301',
 '63339',
 '63967',
 '64327',
 '64476',
 '64771',
 '64832',
 '65334',
 '65547',
 '66030',
 '66049',
 '66058',
 '66068',
 '66087',
 '66750',
 '67263',
 '67299',
 '67335',
 '67353',
 '67857',
 '68005',
 '68409',
 '68460',
 '68512',
 '69632',
 '69660',
 '69687',
 '69911',
 '70119',
 '70847',
 '70868',
 '70878',
 '70888',
 '70892',
 '70909',
 '70914',
 '70973',
 '71001',
 

In [152]:
x_sample = pd.get_dummies(x_sample,columns = ['x1'])


In [151]:
sample_locate = pd.DataFrame(sample_latitude, columns=['fcol_{}'.format(ass)])

KeyboardInterrupt: 

In [110]:
x['x1'] = 10000*(round((x['start_latitude'] * x['start_longitude']),4)-4200)

In [116]:
x['x1'].dtypes

dtype('int32')

In [115]:
x['x1']= x['x1'].astype('int')

In [111]:
A_cnt = x['x1'].value_counts()
print("road_name :\n", x['x1'].value_counts())
print()

road_name :
 82052.0     23291
375458.0    20775
319721.0    18137
405619.0    18084
379927.0    18076
            ...  
428081.0      881
357355.0      788
77807.0       744
83632.0       744
79174.0       587
Name: x1, Length: 586, dtype: int64



##### x_sample['B']  = np.where(x_sample['B'] == 'a', 'aa', x_sample['B'])

In [118]:
ohe = OneHotEncoder()
train_latitude = ohe.fit_transform(x[['x1']])

In [ ]:
s_locate = pd.DataFrame(train_latitude, columns=[col for col in ohe.categories_[0]])

##### for i in df_ho['locdate']:
    x.loc[(x['base_date'] == i),'base_date'] = 1
x.loc[(x['base_date'] > 1),'base_date'] = 0

for i in df_ho['locdate']:
    t_test.loc[(t_test['base_date'] == i),'base_date'] = 1
    
t_test.loc[(t_test['base_date'] > 1),'base_date'] = 0    
x = x.replace('월', 0)
x = x.replace('화', 0)
x = x.replace('수', 0)
x = x.replace('목', 0)
x = x.replace('금', 0)
x = x.replace('토', 1)
x = x.replace('일', 1)
t_test = t_test.replace('월', 0)
t_test = t_test.replace('화', 0)
t_test = t_test.replace('수', 0)
t_test = t_test.replace('목', 0)
t_test = t_test.replace('금', 0)
t_test = t_test.replace('토', 1)
t_test = t_test.replace('일', 1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [77]:
## 훈련데이터세트의 shape ##
print(x.shape)
# (569,30) 

## 훈련세트, 테스트세트 shape ##
print(x_train.shape, x_test.shape)
# (455,30) (114,30)

## 훈련데이터세트의 클래스 비율 ##
np.unique(y, return_counts = True)
# (array([0,1]), array([212,357]))

## 훈련세트의 클래스 비율 ##
np.unique(y_train, return_counts = True)
# (array([0,1]), array([170,285]))

## 훈련세트의 클래스 비율 ##
np.unique(y_test, return_counts = True)
# (array([0,1]), array([42, 72]))

(4701217, 13)
(3760973, 13) (940244, 13)


(array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
        27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.,
        40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52.,
        53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64., 65.,
        66., 67., 68., 69., 70., 71., 72., 73., 74., 75., 76., 77., 78.,
        79., 80., 81., 82., 83., 84., 85., 86., 87., 88., 89., 90., 91.,
        92., 93., 94., 99.]),
 array([   13,    37,    67,   136,   262,   485,   766,  1026,  1433,
         2231,  2645,  3737,  4243,  5134,  6256,  7376,  8889, 10208,
        11071, 11840, 13013, 13406, 14214, 14862, 15329, 15951, 16375,
        17550, 18409, 19268, 19691, 20069, 20536, 18862, 18767, 18855,
        18113, 18710, 18326, 18524, 18595, 18463, 18582, 18707, 19559,
        20115, 21465, 22336, 22063, 21496, 21551, 21307, 21547, 21719,
        20974, 18471, 17351, 1523

##### str_list = ['start_latitude','start_longitude','end_latitude','end_longitude']
for i in str_list:
    if i == 'start_latitude' or i == 'end_latitude':
        X_train[i] -= 33
    else:
        X_train[i] -= 126
for i in str_list:
    if i == 'start_latitude' or i == 'end_latitude':
        test[i] -= 33
    else:
        test[i] -= 126

In [107]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', max_depth = 5)
xg_reg.fit(X_train, y_train)
pred = xg_reg.predict(test)


##### xg_reg = xgb.XGBRegressor(random_state=42).fit(X_train, y_train)

##### pred = xg_reg.predict(test)

In [108]:
sample_submission = pd.read_csv('C:/Users/gisic/gisic/jejutraffic/sample_submission.csv')

In [109]:
sample_submission['target'] = pred
sample_submission.to_csv("C:/Users/gisic/gisic/jejutraffic/submit.csv", index = False)

In [110]:
sample_submission

,id,target
0,TEST_000000,25.886963
1,TEST_000001,46.102360
2,TEST_000002,63.029503
3,TEST_000003,33.488358
4,TEST_000004,39.507397
...,...,...
291236,TEST_291236,52.457191
291237,TEST_291237,52.570896
291238,TEST_291238,22.827877
291239,TEST_291239,24.335571


In [94]:
suspect_submission = sample_submission.copy()

In [95]:
suspect_submission

,id,target
0,TEST_000000,24.408371
1,TEST_000001,45.870052
2,TEST_000002,63.367569
3,TEST_000003,34.505859
4,TEST_000004,37.616901
...,...,...
291236,TEST_291236,51.246002
291237,TEST_291237,52.160847
291238,TEST_291238,21.929148
291239,TEST_291239,25.345930
